# Implementation of Competitor-Aware Shared Vehicle Positioning (CSVP) Framework

In this notebook we provide a Julia-based implementation of the Competitor-aware Shared Vehicle Positioning (CSVP) Framework as developed and outlined in the research paper __"Data-driven competitor-aware positioning in on-demand vehicle rental networks"__. The below basic implementation is intended to facilitate replication and benchmark analyses of the presented research. 

The notebook is structured in a linear manner and contains the following three sections:
1. Parameterizitaion and Data Preparation
2. Optimization Model (as per Eq. (1)-(18) in Section 3.3 of paper)
3. Integrated Simulation Framework (as per Algorithm 1: CSVPSim() in Online Appendix C of paper)

The following libraries are required

In [58]:
using JuMP, Cbc, Ipopt, Juniper, Gurobi # if gurobi is not available use an open source solver (such as CBC) instead
using CSV
using Dates
using PyPlot
using TimeSeries
using DataFrames
using TimerOutputs
using StatsBase
using Random

Data and output directories are defined. Sample data is provided in `../01_Data/`. An output directory has been created at `../03_Output/`.

In [59]:
# define paths variables
data_path = "../01_Data/";
save_path = "../03_Output/_output_files/";

___

### 1. Define core settings and prepare input parameter data

__Define important parameters settings__

In [60]:
spatial_res = 6 # resolution of hexagons
temporal_res = "6h" # resolution of time
optimization_horizon = 3 # in periods

dformat = Dates.DateFormat("yyyy-mm-dd HH:MM:SS");

######################################
# Temporal parameters
test_period = Dates.DateTime("2017-03-13 00:00:00", dformat)
sim_start_period = Dates.DateTime("2017-03-13 00:00:00", dformat)

#Compute timedeltas
if temporal_res == "1h"
    delta_t = Dates.Hour(1)
elseif temporal_res == "6h"
    delta_t = Dates.Hour(6)
end;

sim_end_period = Dates.DateTime("2017-03-20 00:00:00", dformat) + delta_t*4
range_df = CSV.read("$(data_path)A_t_$(spatial_res)hex_$(temporal_res).csv", DataFrame)
range_df[!,4] = Dates.DateTime.(range_df[!,4],dformat)
sim_range = unique(range_df[range_df.timestamp .<= sim_end_period, :][!,4]);
target_period = test_period#sim_start_period

#Compute timedeltas
if temporal_res == "1h"
    delta_t = Dates.Hour(1)
elseif temporal_res == "6h"
    delta_t = Dates.Hour(6)
end;

target_period = test_period+delta_t*2

######################################
# Spatial Parameters

nodes = CSV.read("$(data_path)Regions_$(spatial_res)hex_long.csv", DataFrame)[!,1]
nodes = nodes[1:5]
i_nodes = nodes
j_nodes = nodes

dist_hexes = CSV.read("$(data_path)origin_destination_h$(spatial_res)_haversine.csv", DataFrame)
dist_OSM = CSV.read("$(data_path)origin_destination_h$(spatial_res)_OSM.csv", DataFrame)
dist_h2h = 6.1136 #km per hex (at resolution of 6)

# Cost and revenue Parameters

avg_length_per_rental = 20 #km ---> have taken *2 to account for 
var_cost_pkm = 0.175 # fuel and degradation cost in USD/km
var_cost_pmin = 0.1 # 30km/51min*0.175min
var_cost_per_reloc_pkm = 0.0 # operational relocation cost in USD/km, 0 for autonomous vehicle, 0.7 for human operator

r_rent_avg = 20 #revenue in USD/rent
r_rent_spec = 0.4 #revenue per minute in USD
c_rent_avg = avg_length_per_rental*var_cost_pkm #cost in $/rent;

__Define core input data/paramters__

- I_market_true
- I_market_pred
- I_own_true
- I_own_pred
- D_market_true
- D_market_pred
- A_market_p0
- A_own_p0
- c_reloc
- rent_length

__Load and prepare full datasets__

In [61]:
model_name = "best_model" # "best_model"

I_own_raw_t = CSV.read("$(data_path)C2G_rent_end_$(model_name)_t_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
I_own_raw_t[!,7] = Dates.DateTime.(I_own_raw_t[!,7],dformat)
I_own_raw_t1 = CSV.read("$(data_path)C2G_rent_end_$(model_name)_t+1_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
I_own_raw_t1[!,7] = Dates.DateTime.(I_own_raw_t1[!,7],dformat)
I_own_raw_t2 = CSV.read("$(data_path)C2G_rent_end_$(model_name)_t+2_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
I_own_raw_t2[!,7] = Dates.DateTime.(I_own_raw_t2[!,7],dformat)
I_own_raw_tn = CSV.read("$(data_path)C2G_rent_end_$(model_name)_t+n_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
I_own_raw_tn[!,7] = Dates.DateTime.(I_own_raw_tn[!,7],dformat)

I_market_raw_t = CSV.read("$(data_path)total_rent_end_$(model_name)_t_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
I_market_raw_t[!,7] = Dates.DateTime.(I_market_raw_t[!,7],dformat)
I_market_raw_t1 = CSV.read("$(data_path)total_rent_end_$(model_name)_t+1_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
I_market_raw_t1[!,7] = Dates.DateTime.(I_market_raw_t1[!,7],dformat)
I_market_raw_t2 = CSV.read("$(data_path)total_rent_end_$(model_name)_t+2_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
I_market_raw_t2[!,7] = Dates.DateTime.(I_market_raw_t2[!,7],dformat)
I_market_raw_tn = CSV.read("$(data_path)total_rent_end_$(model_name)_t+n_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
I_market_raw_tn[!,7] = Dates.DateTime.(I_market_raw_tn[!,7],dformat)

D_market_raw_t = CSV.read("$(data_path)total_rent_start_$(model_name)_t_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
D_market_raw_t[!,7] = Dates.DateTime.(D_market_raw_t[!,7],dformat)
D_market_raw_t1 = CSV.read("$(data_path)total_rent_start_$(model_name)_t+1_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
D_market_raw_t1[!,7] = Dates.DateTime.(D_market_raw_t1[!,7],dformat)
D_market_raw_t2 = CSV.read("$(data_path)total_rent_start_$(model_name)_t+2_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
D_market_raw_t2[!,7] = Dates.DateTime.(D_market_raw_t2[!,7],dformat)
D_market_raw_tn = CSV.read("$(data_path)total_rent_start_$(model_name)_t+n_dedicated_results_offline_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
D_market_raw_tn[!,7] = Dates.DateTime.(D_market_raw_tn[!,7],dformat);

A_p0 = CSV.read("$(data_path)A_t_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
A_p0[!,4] = Dates.DateTime.(A_p0[!,4],dformat);

__Inflows and Outflows__

- I_own
- I_market
- D_market

In [62]:
# Simulation inputs

I_own_true = Dict()
for i in nodes #range(1,stop=length(nodes))
    for t in sim_range #range(1,stop=length(sim_range))
        I_own_true[i,t] = I_own_raw_t[(I_own_raw_t.timestamp .== t).& (I_own_raw_t[:,6].== i), :][1,4]
    end
end

I_market_true = Dict()
for i in nodes #range(1,stop=length(nodes))
    for t in sim_range #range(1,stop=length(sim_range))
        I_market_true[i,t] = I_market_raw_t[(I_market_raw_t.timestamp .== t).& (I_market_raw_t[:,6].== i), :][1,4]
    end
end

D_market_true = Dict()
for i in nodes #range(1,stop=length(nodes))
    for t in sim_range #range(1,stop=length(sim_range))
        D_market_true[i,t] = D_market_raw_t[(D_market_raw_t.timestamp .== t).& (D_market_raw_t[:,6].== i), :][1,4]
    end
end

In [63]:
# Optimization input (goes into inside loop of simulation as it will need to be refreshed every period)
horizon_periods =4
times_max = [target_period, target_period+delta_t, target_period+2*delta_t, target_period+3*delta_t,target_period+4*delta_t];
horizon_t = times_max[1:horizon_periods];

I_own_true_MOLO = Dict()
for i in nodes
    I_own_true_MOLO[i,horizon_t[1]] = I_own_raw_t[(I_own_raw_t.timestamp .== horizon_t[1]).& (I_own_raw_t[:,6] .== i), :][1,4]
    I_own_true_MOLO[i,horizon_t[2]] = I_own_raw_t1[(I_own_raw_t1.timestamp .== horizon_t[2]).& (I_own_raw_t1[:,6] .== i), :][1,4]
    I_own_true_MOLO[i,horizon_t[3]] = I_own_raw_t2[(I_own_raw_t2.timestamp .== horizon_t[3]).& (I_own_raw_t2[:,6] .== i), :][1,4]
    I_own_true_MOLO[i,horizon_t[4]] = I_own_raw_tn[(I_own_raw_tn.timestamp .== horizon_t[4]).& (I_own_raw_tn[:,6] .== i), :][1,4]
end

I_own_pred_MOLO = Dict()
for i in nodes
    I_own_pred_MOLO[i,horizon_t[1]] = I_own_raw_t[(I_own_raw_t.timestamp .== horizon_t[1]).& (I_own_raw_t[:,6] .== i), :][1,5]
    I_own_pred_MOLO[i,horizon_t[2]] = I_own_raw_t1[(I_own_raw_t1.timestamp .== horizon_t[2]).& (I_own_raw_t1[:,6] .== i), :][1,5]
    I_own_pred_MOLO[i,horizon_t[3]] = I_own_raw_t2[(I_own_raw_t2.timestamp .== horizon_t[3]).& (I_own_raw_t2[:,6] .== i), :][1,5]
    I_own_pred_MOLO[i,horizon_t[4]] = I_own_raw_tn[(I_own_raw_tn.timestamp .== horizon_t[4]).& (I_own_raw_tn[:,6] .== i), :][1,5]
end

I_market_true_MOLO = Dict()
for i in nodes
    I_market_true_MOLO[i,horizon_t[1]] = I_market_raw_t[(I_market_raw_t.timestamp .== horizon_t[1]).& (I_market_raw_t[:,6] .== i), :][1,4]
    I_market_true_MOLO[i,horizon_t[2]] = I_market_raw_t1[(I_market_raw_t1.timestamp .== horizon_t[2]).& (I_market_raw_t1[:,6] .== i), :][1,4]
    I_market_true_MOLO[i,horizon_t[3]] = I_market_raw_t2[(I_market_raw_t2.timestamp .== horizon_t[3]).& (I_market_raw_t2[:,6] .== i), :][1,4]
    I_market_true_MOLO[i,horizon_t[4]] = I_market_raw_tn[(I_own_raw_tn.timestamp .== horizon_t[4]).& (I_market_raw_tn[:,6] .== i), :][1,4]
end

I_market_pred_MOLO = Dict()
for i in nodes
    I_market_pred_MOLO[i,horizon_t[1]] = I_market_raw_t[(I_market_raw_t.timestamp .== horizon_t[1]).& (I_market_raw_t[:,6] .== i), :][1,5]
    I_market_pred_MOLO[i,horizon_t[2]] = I_market_raw_t1[(I_market_raw_t1.timestamp .== horizon_t[2]).& (I_market_raw_t1[:,6] .== i), :][1,5]
    I_market_pred_MOLO[i,horizon_t[3]] = I_market_raw_t2[(I_market_raw_t2.timestamp .== horizon_t[3]).& (I_market_raw_t2[:,6] .== i), :][1,5]
    I_market_pred_MOLO[i,horizon_t[4]] = I_market_raw_tn[(I_own_raw_tn.timestamp .== horizon_t[4]).& (I_market_raw_tn[:,6] .== i), :][1,5]
end

D_market_true_MOLO = Dict()
for i in nodes
    D_market_true_MOLO[i,horizon_t[1]] = D_market_raw_t[(D_market_raw_t.timestamp .== horizon_t[1]).& (D_market_raw_t[:,6] .== i), :][1,4]
    D_market_true_MOLO[i,horizon_t[2]] = D_market_raw_t1[(D_market_raw_t1.timestamp .== horizon_t[2]).& (D_market_raw_t1[:,6] .== i), :][1,4]
    D_market_true_MOLO[i,horizon_t[3]] = D_market_raw_t2[(D_market_raw_t2.timestamp .== horizon_t[3]).& (D_market_raw_t2[:,6] .== i), :][1,4]
    D_market_true_MOLO[i,horizon_t[4]] = D_market_raw_tn[(D_market_raw_tn.timestamp .== horizon_t[4]).& (D_market_raw_tn[:,6] .== i), :][1,4]
end

D_market_pred_MOLO = Dict()
for i in nodes
    D_market_pred_MOLO[i,horizon_t[1]] = D_market_raw_t[(D_market_raw_t.timestamp .== horizon_t[1]).& (D_market_raw_t[:,6] .== i), :][1,5]
    D_market_pred_MOLO[i,horizon_t[2]] = D_market_raw_t1[(D_market_raw_t1.timestamp .== horizon_t[2]).& (D_market_raw_t1[:,6] .== i), :][1,5]
    D_market_pred_MOLO[i,horizon_t[3]] = D_market_raw_t2[(D_market_raw_t2.timestamp .== horizon_t[3]).& (D_market_raw_t2[:,6] .== i), :][1,5]
    D_market_pred_MOLO[i,horizon_t[4]] = D_market_raw_tn[(D_market_raw_tn.timestamp .== horizon_t[4]).& (D_market_raw_tn[:,6] .== i), :][1,5]
end

__Observed vehicle availabilities__

- A_market
- A_own
- A_own_max_i

In [64]:
A_market_p0 = Dict()
for i in nodes #range(1,stop=length(nodes))
    for t in sim_range #range(1,stop=length(sim_range))
        A_market_p0[i,t] = round(A_p0[(A_p0.timestamp .== t).&(A_p0[:,3] .== i), :][1,9],digits=0)
    end
end

A_own_p0 = Dict()
for i in nodes #range(1,stop=length(nodes))
    for t in sim_range #range(1,stop=length(sim_range))
        A_own_p0[i,t] = round(A_p0[(A_p0.timestamp .== t).&(A_p0[:,3] .== i), :][1,5]+A_p0[(A_p0.timestamp .== t).&(A_p0[:,3] .== i), :][1,6],digits=0)
    end
end


# define max values (time independent)

A_own_max = Dict()
for i in nodes
    A_own_max[i] = round(maximum(A_p0[A_p0[!,3] .== i,:][!,6]),digits=0)
end

I_own_max = Dict()
for i in nodes
    I_own_max[i] = round(maximum(I_own_raw_t[I_own_raw_t[!,6] .== i,:][!,4]),digits=0)
end

H_max = Dict()
for i in nodes
    H_max[i] = round(maximum(A_p0[!,9])*1.5,digits=0) #[A_p0[!,3] .== i,:]
end

__Re-loc cost__
- c_reloc

In [65]:
# compute cost per reloc and safe in dictionary

c_reloc_haversine = Dict()
for i in nodes
    for j in nodes
        c_reloc_haversine[i,j] =  float(dist_hexes[(dist_hexes.h_orgin .== i).&(dist_hexes.h_destination .==j),:][:,9][1]) *dist_h2h* (var_cost_pkm+var_cost_per_reloc_pkm)# c is distance in terms of nuber of hex-tiles
    end
end

c_reloc_OSM = Dict()
for i in nodes
    for j in nodes
        c_reloc_OSM[i,j] =  float(dist_OSM[(dist_OSM.h_orgin .== i).&(dist_OSM.h_destination .==j),:][:,8][1]) * (var_cost_pkm+var_cost_per_reloc_pkm)# c is distance in terms of nuber of hex-tiles
    end
end


__Rental length parameters__
- rent_length

In [66]:
Length = CSV.read("$(data_path)Length_true_$(spatial_res)hex_$(temporal_res).csv",DataFrame)
Length[!,11] = Dates.DateTime.(Length_true[!,11],dformat);

Length_own=Dict()
for i in nodes, t in sim_range
    if size(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:],1) > 0
        Length_own[i,t] = round(float(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:][1,8]),digits=2)  
    else
        Length_own[i,t] = 0
    end
end

Length_comp=Dict()
for i in nodes, t in sim_range
    if size(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:],1) > 0
        Length_comp[i,t] = round(float(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:][1,5]),digits=2)  
    else
        Length_comp[i,t] = 0
    end
end

Length_avg=Dict()
for i in nodes, t in sim_range
    if size(Length[(Length_true.timestamp .== t).&(Length[!,10] .== i),:],1) > 0
        Length_avg[i,t] = round(float(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:][1,9]),digits=2)  
    else
        Length_avg[i,t] = 0
    end
end    

___

### 2. Optimization Model:

Implementation of dyamic C-SVAP as MINLP using the avg. rental length as additional factor in objective. We use juniper: https://lanl-ansi.github.io/Juniper.jl/v0.1/options.html as MINLP solver. The optimization draws on the previously computed input parameters.

In [68]:
function CSVP_optimization(target_period, horizon_num_periods, delta_t, i_nodes, j_nodes,
        D_market,A_market,I_market,A_own,I_own,length_avg_min,H_max,M_max,r_rent_pmin,c_rent_pmin,c_reloc)
    
    ################ PRE-COMPUTE SELCTED PARAMS ################ 
    # define set of times
    
    times_max = [target_period, target_period+delta_t, target_period+2*delta_t, target_period+3*delta_t,target_period+4*delta_t]
    times = times_max[1:horizon_num_periods]
    
    # pre-compute base availability (no relocations)
    
    for i in i_nodes, t in times[2:length(times)]
        A_own[i,t] = round(A_own[i,t-delta_t] + I_own[i,t-delta_t] - D_market[i,t-delta_t] * ((A_own[i,t-delta_t] + I_own[i,t-delta_t]) / (A_market[i,t-delta_t] + I_market[i,t-delta_t])),digits=0)
        A_market[i,t] = round(A_market[i,t-delta_t] + I_market[i,t-delta_t] - D_market[i,t-delta_t])
        
        #ensure that market availability is not exceeded
        if A_own[i,t]>A_market[i,t]
            A_own[i,t] == A_market[i,t]
        end
    end
    

    
    ################ MODEL ################
    
    #optimizers
    optimizer = Juniper.Optimizer
    nl_solver= optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0, "max_cpu_time" => 900.0, "max_iter" => 100000)
    #mip_solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 0)
    mip_solver = optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0,"TimeLimit"=>15*60, "Threads"=>4) ## 1.8 GHz Intel Core i5 has 1 processor, 2 cores and 4 thread
    
    C_SVAP_MINLP_DYNAMIC = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver, "mip_solver"=>mip_solver,
            "time_limit" => 900.00, "feasibility_pump" => true, "allow_almost_solved_integral" => true, "log_levels" => [],
            "atol"=>0.01, "mip_gap"=>0.02)) #log levels [:Table,:Info]
    
    
    ################ VARIABLES ################

    # Decision variable , incl Contraint 1:
    @variable(C_SVAP_MINLP_DYNAMIC, x[i_nodes, j_nodes, times] >= 0, Int)

    # avoid circular re-locations (not strictly needed)
    for i in i_nodes, t in times
      @constraint(C_SVAP_MINLP_DYNAMIC, sum(x[i,i,t]) == 0)
    end

    # State variables (also incl. as variables on which we impose constraints)
    @variable(C_SVAP_MINLP_DYNAMIC, A_own_state[i_nodes,times])
    @variable(C_SVAP_MINLP_DYNAMIC, A_market_state[i_nodes, times])
    @variable(C_SVAP_MINLP_DYNAMIC, I_own_state[i_nodes, times])
    
    @variable(C_SVAP_MINLP_DYNAMIC, D_market_state[i_nodes, times] >= 0) #needed to constrain demand to actual availability in tile

    
    ################ OBJECTIVE ################
    
    @NLobjective(C_SVAP_MINLP_DYNAMIC, Max, sum(sum(D_market_state[i,t] * ((A_own_state[i,t] + I_own_state[i,t] + sum(x[j,i,t] - x[i,j,t] for j in j_nodes)) / (A_market_state[i,t] + I_market[i,t] + sum(x[j,i,t] - x[i,j,t] for j in j_nodes))) * length_avg_min[i,t] * (r_rent_pmin - c_rent_pmin) for i in i_nodes) - sum(x[i,j,t]*c_reloc[i,j] for i in i_nodes, j in j_nodes) for t in times))
    
    
    ################ CONSTRAINTS ################
    
    # Constraint 2: Cannot re-locate more than available throughout period
    for i in i_nodes, t in times
        @constraint(C_SVAP_MINLP_DYNAMIC, sum(x[i,j,t]-x[j,i,t] for j in j_nodes) <= (A_own_state[i,t])) #+I_own_state[i,t]
    end
    
    #Constraint 3 a+b: Limit demand to actual availability 
    for i in i_nodes, t in times
        @constraint(C_SVAP_MINLP_DYNAMIC,  D_market_state[i,t]<= D_market[i,t])
    end
    
    for i in i_nodes, t in times
        @constraint(C_SVAP_MINLP_DYNAMIC,  D_market_state[i,t]<= A_market_state[i,t] + I_market[i,t] + sum(x[j,i,t] - x[i,j,t] for j in j_nodes))
    end
    
    #for i in i_nodes, t in times
    #    @constraint(C_SVAP_MINLP_DYNAMIC, (A_market_state[i,t] + I_market[i,t] + sum(x[j,i,t] - x[i,j,t] for j in j_nodes)) >= D_market[i,t])
    #end
    
    # Constraint 4: Do not add more cars to tile than capacity within tile
    for i in i_nodes, t in times
        @constraint(C_SVAP_MINLP_DYNAMIC, (A_market_state[i,t] + I_market[i,t] + sum(x[j,i,t] - x[i,j,t] for j in j_nodes) - D_market_state[i,t]) <= H_max[i])
    end
    
     # Constraint 5: Limit re-locations per period
    for t in times
        @constraint(C_SVAP_MINLP_DYNAMIC, sum(x[i,j,t] for j in j_nodes, i in i_nodes) <= M_max)
    end

    # Constraints 5ff: Initialize and update the state variables
    # INITIALIZE
    for i in i_nodes
        @constraint(C_SVAP_MINLP_DYNAMIC, A_market_state[i,times[1]] == A_market[i,times[1]])
    end

    for i in i_nodes
        @constraint(C_SVAP_MINLP_DYNAMIC, A_own_state[i,times[1]] == A_own[i,times[1]])
    end
    
    for i in i_nodes
        @constraint(C_SVAP_MINLP_DYNAMIC, I_own_state[i,times[1]] == I_own[i,times[1]])
    end

    #UPDATE
    # need to use -delta_t to indicate last period

    for i in i_nodes, t in times[2:length(times)]
        @constraint(C_SVAP_MINLP_DYNAMIC, A_market_state[i,t] == A_market_state[i,t-delta_t] + I_market[i,t-delta_t] - D_market_state[i,t-delta_t] + sum(x[j,i,t-delta_t] - x[i,j,t-delta_t] for j in j_nodes) )
    end

    for i in i_nodes, t in times[2:length(times)]
        @NLconstraint(C_SVAP_MINLP_DYNAMIC, A_own_state[i,t] == A_own_state[i,t-delta_t] + I_own_state[i,t-delta_t] + sum(x[j,i,t-delta_t] - x[i,j,t-delta_t] for j in j_nodes) - D_market_state[i,t-delta_t] * ((A_own_state[i,t-delta_t] + I_own_state[i,t-delta_t] + sum(x[j,i,t-delta_t] - x[i,j,t-delta_t] for j in j_nodes)) / (A_market_state[i,t-delta_t] + I_market[i,t-delta_t] + sum(x[j,i,t-delta_t] - x[i,j,t-delta_t] for j in j_nodes))))
    end
    
    for i in i_nodes, t in times[2:length(times)]
        @NLconstraint(C_SVAP_MINLP_DYNAMIC, I_own_state[i,t] == I_own[i,t] +  (((A_own_state[i,t-delta_t] + I_own_state[i,t-delta_t] + sum(x[j,i,t-delta_t] - x[i,j,t-delta_t] for j in j_nodes)) / (A_market_state[i,t-delta_t] + I_market[i,t-delta_t] + sum(x[j,i,t-delta_t] - x[i,j,t-delta_t] for j in j_nodes))) - (A_own[i,t-delta_t] + I_own[i,t-delta_t]) / (A_market[i,t-delta_t] + I_market[i,t-delta_t])) * D_market_state[i,t-delta_t] * (I_market[i,t]/ sum(I_market[j,t] for j in j_nodes))) 
    end

    ################ RUN MODEL ################
    
    #print(C_SVAP_MINLP_DYNAMIC)
    optimize!(C_SVAP_MINLP_DYNAMIC)
    
    ################ COLLECT RESULTS ################
    
    X_dict_MINLP_DYNAMIC = Dict()
    for i in i_nodes
        for j in j_nodes
            for t in times
                X_dict_MINLP_DYNAMIC[i,j,t] = round(JuMP.value(x[i,j,t]))
            end
        end
    end
    
    D_dict_MINLP_DYNAMIC = Dict()
    for i in i_nodes, t in times
        D_dict_MINLP_DYNAMIC[i,t] = round(JuMP.value(D_market_state[i,t]))
    end

    return  X_dict_MINLP_DYNAMIC
end

CSVP_optimization (generic function with 1 method)

In [73]:
active=false

if active
    M_max=300
    r_rent_pmin = r_rent_spec
    c_rent_pmin = var_cost_pmin
    X_dict_trial_3 = @time CSVP_optimization(target_period, 1, delta_t, i_nodes, j_nodes,
                    D_market_true_MOLO,A_market_p0,I_market_true_MOLO,A_own_p0,I_own_true_MOLO,
                    Length_avg,H_max,M_max,r_rent_pmin,c_rent_pmin,c_reloc_OSM);

LoadError: [91msyntax: incomplete: premature end of input[39m

___

### 3. Simulation

The below code implements a discrete event simulation of the CSVP optimization as laid out by Algorithm 1 (Online Appedix C)

In [71]:
function CSVPSim(suffix)

    
"""
suffix = str to be added at end of output files, 
"""
    
    # define paths
    data_path = "../01_Data/";
    save_path = "../03_Output/_output_files/";
    
    # sensitivity factors are switched off in this code sample!
    demand_factor = 0
    added_supply_comp = 0
    added_supply_own = 0
    outage_probability = 0
    response_strat = "DONOTHING"
 
    dformat = Dates.DateFormat("yyyy-mm-dd HH:MM:SS")
    
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
    #### Define constant cost and revenue paramters   
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##

    avg_length_per_rental = 20 #km --> conservative!
    var_cost_pkm = 0.175 #0.175 # fuel and degradation cost in USD/km
    var_cost_pmin = 0.07 # 20km/51min*0.175 = 0.045; 15km/51min*0.2min = 0.06
    
    var_cost_per_reloc_pkm_AUTO = 0
    var_cost_per_reloc_pkm_HUM = 0.7
    var_cost_per_reloc_pkm_HYBRID = var_cost_per_reloc_pkm_HUM/2
    
    # probability of outage
    outage_probability = 0
    
    r_rent_avg = 20 # revenue in USD/rent
    r_rent_spec = 0.4 # revenue per minute in USD
    c_rent_avg = avg_length_per_rental*var_cost_pkm #cost in $/rent;
    
    S_market= 2285 # check!
    S_own=988
    M_max = 300 # max number of relocations per period
    
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
    #### Loop over spatial and temporal res
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
    
    for temporal_res in ["1h"] # data for "1h" and "6h" provided
        for hex_res in [6] # data for hex_res=6 provided     
            
            if hex_res ==5 
                num_regions = 5
            elseif hex_res ==6 
                num_regions = 13
            elseif hex_res ==7 
                num_regions = 43
            end

            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
            #### Define temporal parameters    
            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##

            #Temporal Parameters
            if temporal_res == "1h"
                delta_t = Dates.Hour(1)
            elseif temporal_res == "6h"
                delta_t = Dates.Hour(6)
            end;
            
            # 7 day simulation period
            start_period = Dates.DateTime("2017-03-13 00:00:00", dformat)
            end_period = Dates.DateTime("2017-03-20 00:00:00", dformat)-delta_t

            sim_range = []
            iterator = start_period
            while iterator <= end_period
                push!(sim_range,iterator)
                iterator+=delta_t 
            end
            
            # randomly sample outage periods
            outage_periods = randsubseq(sim_range,outage_probability)
            println(length(outage_periods)," outage periods selected")

            #println(sim_range)

            data_range = []
            iterator = start_period
            while iterator <= end_period+delta_t*3
                push!(data_range,iterator)
                iterator+=delta_t 
            end

            #println(data_range)
            
            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
            #### Loop over models, info types and re-loc types
            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
            for model in ["CSVP-2"] # "CSVP-1","CSVP-2"
                for info in ["PI"]  # "PRED","PI"
                    for reloc_type in ["HUMAN","AUTO","HYBRID"] # "HUMAN","AUTO","HYBRID"
                    
                        
                        println("$hex_res, $temporal_res ,$model, $info, $reloc_type started")
                        
                        infeasibility_checker = Dict()
                        opt_timer = Dict()

                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### Load sim-specific data
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##

                        dformat = Dates.DateFormat("yyyy-mm-dd HH:MM:SS")

                        nodes = CSV.read("$(data_path)Regions_$(hex_res)hex_long.csv",DataFrame)[!,1]
                        nodes = nodes[1:num_regions]
                        
                        println("Number of regions: $(length(nodes))")
                    
                        dist_hexes = CSV.read("$(data_path)origin_destination_h$(hex_res)_haversine.csv",DataFrame)
                        dist_OSM = CSV.read("$(data_path)origin_destination_h$(hex_res)_OSM.csv",DataFrame)
                        
                        model_name = "LinearRegression" #alternatively set this to LinearRegression
                        
                        I_own_raw_t = CSV.read("$(data_path)C2G_rent_end_$(model_name)_t_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        I_own_raw_t[!,7] = Dates.DateTime.(I_own_raw_t[!,7],dformat)
                        I_own_raw_t1 = CSV.read("$(data_path)C2G_rent_end_$(model_name)_t+1_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        I_own_raw_t1[!,7] = Dates.DateTime.(I_own_raw_t1[!,7],dformat)
                        I_own_raw_t2 = CSV.read("$(data_path)C2G_rent_end_$(model_name)_t+2_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        I_own_raw_t2[!,7] = Dates.DateTime.(I_own_raw_t2[!,7],dformat)
                        I_own_raw_tn = CSV.read("$(data_path)C2G_rent_end_$(model_name)_t+n_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        I_own_raw_tn[!,7] = Dates.DateTime.(I_own_raw_tn[!,7],dformat)

                        I_market_raw_t = CSV.read("$(data_path)total_rent_end_$(model_name)_t_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        I_market_raw_t[!,7] = Dates.DateTime.(I_market_raw_t[!,7],dformat)
                        I_market_raw_t1 = CSV.read("$(data_path)total_rent_end_$(model_name)_t+1_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        I_market_raw_t1[!,7] = Dates.DateTime.(I_market_raw_t1[!,7],dformat)
                        I_market_raw_t2 = CSV.read("$(data_path)total_rent_end_$(model_name)_t+2_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        I_market_raw_t2[!,7] = Dates.DateTime.(I_market_raw_t2[!,7],dformat)
                        I_market_raw_tn = CSV.read("$(data_path)total_rent_end_$(model_name)_t+n_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        I_market_raw_tn[!,7] = Dates.DateTime.(I_market_raw_tn[!,7],dformat)

                        D_market_raw_t = CSV.read("$(data_path)total_rent_start_$(model_name)_t_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        D_market_raw_t[!,7] = Dates.DateTime.(D_market_raw_t[!,7],dformat)
                        D_market_raw_t1 = CSV.read("$(data_path)total_rent_start_$(model_name)_t+1_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        D_market_raw_t1[!,7] = Dates.DateTime.(D_market_raw_t1[!,7],dformat)
                        D_market_raw_t2 = CSV.read("$(data_path)total_rent_start_$(model_name)_t+2_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        D_market_raw_t2[!,7] = Dates.DateTime.(D_market_raw_t2[!,7],dformat)
                        D_market_raw_tn = CSV.read("$(data_path)total_rent_start_$(model_name)_t+n_dedicated_results_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        D_market_raw_tn[!,7] = Dates.DateTime.(D_market_raw_tn[!,7],dformat);

                        A_p0 = CSV.read("$(data_path)A_t_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        A_p0[!,4] = Dates.DateTime.(A_p0[!,4],dformat)
                        Length = CSV.read("$(data_path)Length_true_$(hex_res)hex_$(temporal_res).csv",DataFrame)
                        Length[!,11] = Dates.DateTime.(Length_true[!,11],dformat);
                        
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### Prepare dictionaries for observable data
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        
                        # True values
                        D_market_true_t = Dict()
                        D_market_pred_t = Dict()
                        D_market_pred_t1 = Dict()
                        D_market_pred_t2 = Dict()
                        D_market_pred_tn = Dict()
                        
                        I_market_true_t = Dict()
                        I_market_pred_t = Dict()
                        I_market_pred_t1 = Dict()
                        I_market_pred_t2 = Dict()
                        I_market_pred_tn = Dict()
                        
                        I_own_true_t = Dict()
                        I_own_pred_t = Dict()
                        I_own_pred_t1 = Dict()
                        I_own_pred_t2 = Dict()
                        I_own_pred_tn = Dict()
                        A_market_obs = Dict()
                        A_own_obs = Dict()
                        H_max = Dict()    

                        for i in nodes, t in data_range
                            
                            D_market_true_t[i,t] = D_market_raw_t[(D_market_raw_t.timestamp .== t).& (D_market_raw_t[:,6].== i), :][1,4]
                            D_market_pred_t[i,t] = D_market_raw_t[(D_market_raw_t.timestamp .== t).& (D_market_raw_t[:,6].== i), :][1,5]
                            D_market_pred_t1[i,t] = D_market_raw_t1[(D_market_raw_t1.timestamp .== t).& (D_market_raw_t1[:,6].== i), :][1,5]
                            D_market_pred_t2[i,t] = D_market_raw_t2[(D_market_raw_t2.timestamp .== t).& (D_market_raw_t2[:,6].== i), :][1,5]
                            D_market_pred_tn[i,t] = D_market_raw_tn[(D_market_raw_tn.timestamp .== t).& (D_market_raw_tn[:,6].== i), :][1,5]

                            I_market_true_t[i,t] = I_market_raw_t[(I_market_raw_t.timestamp .== t).& (I_market_raw_t[:,6].== i), :][1,4]
                            I_market_pred_t[i,t] = I_market_raw_t[(I_market_raw_t.timestamp .== t).& (I_market_raw_t[:,6].== i), :][1,5]
                            I_market_pred_t1[i,t] = I_market_raw_t1[(I_market_raw_t1.timestamp .== t).& (I_market_raw_t1[:,6].== i), :][1,5]
                            I_market_pred_t2[i,t] = I_market_raw_t2[(I_market_raw_t2.timestamp .== t).& (I_market_raw_t2[:,6].== i), :][1,5]
                            I_market_pred_tn[i,t] = I_market_raw_tn[(I_market_raw_tn.timestamp .== t).& (I_market_raw_tn[:,6].== i), :][1,5]

                            I_own_true_t[i,t] = I_own_raw_t[(I_own_raw_t.timestamp .== t).& (I_own_raw_t[:,6].== i), :][1,4]
                            I_own_pred_t[i,t] = I_own_raw_t[(I_own_raw_t.timestamp .== t).& (I_own_raw_t[:,6].== i), :][1,5]
                            I_own_pred_t1[i,t] = I_own_raw_t1[(I_own_raw_t1.timestamp .== t).& (I_own_raw_t1[:,6].== i), :][1,5]
                            I_own_pred_t2[i,t] = I_own_raw_t2[(I_own_raw_t2.timestamp .== t).& (I_own_raw_t2[:,6].== i), :][1,5]
                            I_own_pred_tn[i,t] = I_own_raw_tn[(I_own_raw_tn.timestamp .== t).& (I_own_raw_tn[:,6].== i), :][1,5]
                            
                            A_market_obs[i,t] = A_p0[(A_p0.timestamp .== t).&(A_p0[:,3] .== i), :][1,9]                            
                            A_own_obs[i,t] = A_p0[(A_p0.timestamp .== t).&(A_p0[:,3] .== i), :][1,5]+A_p0[(A_p0.timestamp .== t).&(A_p0[:,3] .== i), :][1,6]
                            
                            H_max[i] = maximum(A_p0[!,9])*2 #[A_p0[!,3] .== i,:]
 
                        end
                        
                        
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### Compute length parameters
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        
                        Length_own=Dict()
                        Length_comp=Dict()
                        Length_avg=Dict()
                        
                        for i in nodes, t in data_range  
                            if size(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:],1) > 0
                                Length_own[i,t] = round(float(Length_true[(Length.timestamp .== t).&(Length[!,10] .== i),:][1,8]),digits=2)  
                            else
                                Length_own[i,t] = 0
                            end
                            
                            if size(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:],1) > 0
                                Length_comp[i,t] = round(float(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:][1,5]),digits=2)  
                            else
                                Length_comp[i,t] = 0
                            end
                            
                            if size(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:],1) > 0
                                Length_avg[i,t] = round(float(Length[(Length.timestamp .== t).&(Length[!,10] .== i),:][1,9]),digits=2)  
                            else
                                Length_avg[i,t] = 0
                            end
                        end
                        
                        
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### Initialize case-specific parameters and collection dictionaries
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        
                        # set optimization horizon depending on model choice
                        if model == "CSVP-1"
                            horizon_num_periods = 1
                        elseif model == "CSVP-2"
                            horizon_num_periods = 2
                        elseif model == "CSVP-3"
                            horizon_num_periods = 3
                        elseif model == "CSVP-4"
                            horizon_num_periods = 4
                        else
                            horizon_num_periods = 1
                        end
                        
                        # set hex-to-hex distance depending on resolution choice
                        if hex_res == 6
                            dist_h2h = 6.1136 #km per hex
                        elseif hex_res ==7
                            dist_h2h = 2.3077 #km per hex
                        elseif hex_res ==8
                            dist_h2h = 0.87229 #km per hex
                        elseif hex_res == 5
                            dist_h2h = 16.155606578484345
                        end

                        # set var reloc cost depending on reloc_type
                        if reloc_type == "AUTO"
                            var_cost_per_reloc_pkm = var_cost_per_reloc_pkm_AUTO
                        elseif reloc_type == "HUMAN"
                            var_cost_per_reloc_pkm = var_cost_per_reloc_pkm_HUM
                        elseif reloc_type == "HYBRID"
                            var_cost_per_reloc_pkm = var_cost_per_reloc_pkm_HYBRID
                        end
                        
                        
                        # compute cost dictionaries (haversine and OSM)
                        c_reloc_haversine = Dict()
                        for i in nodes, j in nodes
                            c_reloc_haversine[i,j] =  round(float(dist_hexes[(dist_hexes.h_orgin .== i).&(dist_hexes.h_destination .==j),:][:,9][1]) *dist_h2h* (var_cost_pkm+(var_cost_per_reloc_pkm)),digits=2) # c is distance in terms of nuber of hex-tiles
                        end

                        c_reloc_OSM = Dict()
                        for i in nodes, j in nodes
                            c_reloc_OSM[i,j] =  round(float(dist_OSM[(dist_OSM.h_orgin .== i).&(dist_OSM.h_destination .==j),:][:,8][1]) * (var_cost_pkm+(var_cost_per_reloc_pkm)),digits=2)# c is distance in terms of nuber of hex-tiles
                        end
                        
                        
                        # set here which re_loc cost is wanted
                        c_reloc = c_reloc_OSM # use actual open streetmap distances
                        c_var = var_cost_pmin # var cost per min rented
                        r_var = r_rent_spec  # revenue per min rented
                        r_rent_avg = r_rent_avg # avg revenue per rental
                        c_rent_avg = c_rent_avg # avg cost per rental

                        # pre_define dictionaries that will be filled by loop
                        
                        s_own_pre_reloc = Dict()
                        s_own_post_reloc = Dict()
                        
                        D_own_pre_reloc = Dict() # collect own rentals pre re-loc (based on share)
                        D_comp_pre_reloc = Dict() # collect comp. revenue with re-locations
                        D_own_post_reloc = Dict() # collect own rentals post re-loc (based on share)
                        D_market_lost_no_reloc = Dict() # Collect lost demand due to insufficient availability
                        D_market_lost = Dict() # Collect lost demand due to insufficient availability

                        R_own_pre_reloc = Dict() # collect own revenue without re-locations
                        R_comp_pre_reloc = Dict() # collect comp. revenue without re-locations
                        R_own_post_reloc = Dict() # collect own revenue with re-locations

                        C_own_pre_reloc = Dict() # collect own cost for rentals without re-locations
                        C_comp_pre_reloc = Dict() # collect comp. cost for rentals without re-locations
                        C_own_post_reloc = Dict() # collect own cost for rentals with re-locations
                        
                        X_costs = Dict() # collects costs per re-location (initilize to 0 for sim_start-1)

                        # initialize dictionaries where needed
                        X = Dict()  # re-location decision (initilize to 0 for sim_start-1)
                        for i in nodes, j in nodes
                                X[i,j,start_period-delta_t] = 0
                        end

                        Delta_D_own = Dict() # collects D_own post re-location 
                        for i in nodes
                            Delta_D_own[i,start_period-delta_t] = 0
                        end
                        
                        Cum_relocs = Dict()  # tracks cum. avail change 
                        for i in nodes
                            Cum_relocs[i,start_period-delta_t] = 0
                        end
                        
                        Cum_Delta_D_own = Dict()  # tracks cum. avail change 
                        for i in nodes
                            Cum_Delta_D_own[i,start_period-delta_t] = 0
                        end
                        
                        
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### COMPUTE BASE VALUES FOR DEMAND AND INFLOWS (incl. added demand but at existing supply shares)
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        
                        
                        # simulate base own demand values (at constant share incl demand uplift) over data range
                        A_tracker_market_base = A_market_obs
                        A_tracker_own_base = A_own_obs
                        D_market_true_base_t = Dict()
                        D_own_true_base_t = Dict()
                        D_delta_market_base = Dict()
                        D_delta_own_base = Dict()
                        s_own_tracker_t = Dict()
                             
                        # initialize for first period
                        I_market_true_base_t=Dict()
                        I_own_true_base_t = Dict()
                        
                        for i in nodes
                            I_market_true_base_t[i,data_range[1]] = round(I_market_true_t[i,data_range[1]] * (1+demand_factor),digits=0)
                            I_own_true_base_t[i,data_range[1]] = round(I_own_true_t[i,data_range[1]] * (1+demand_factor),digits=0)
                        end
       
                        for t in data_range[1:length(data_range)-1] 
                            for i in nodes
                                
                                # increase by factor
                                D_market_true_base_t[i,t] = round(D_market_true_t[i,t] * (1+demand_factor),digits =0)
                               
                                if D_market_true_base_t[i,t] > A_tracker_market_base[i,t]+I_market_true_base_t[i,t]
                                    D_market_true_base_t[i,t] = A_tracker_market_base[i,t]+I_market_true_base_t[i,t]
                                end
                                
                                D_delta_market_base[i,t] = D_market_true_base_t[i,t]- round(D_market_true_t[i,t] ,digits =0) #* (1+demand_factor)
                                
                                s_own_tracker_t[i,t] = ((A_tracker_own_base[i,t]+I_own_true_base_t[i,t])/(A_tracker_market_base[i,t]+I_market_true_base_t[i,t]))
                                if (A_tracker_market_base[i,t]+I_market_true_base_t[i,t]) == 0
                                    s_own_tracker_t[i,t] = 0
                                end

                                D_own_true_base_t[i,t] = round(round(D_market_true_t[i,t] * (1+demand_factor),digits =0) * s_own_tracker_t[i,t],digits=0)
                                
                                if D_own_true_base_t[i,t] > A_tracker_own_base[i,t]+I_own_true_base_t[i,t]
                                    D_own_true_base_t[i,t] = A_tracker_own_base[i,t]+I_own_true_base_t[i,t]
                                end

                                D_delta_own_base[i,t]  = D_own_true_base_t[i,t] - round(D_market_true_t[i,t] * s_own_tracker_t[i,t],digits=0)   #D_market_true_base_t
                                
                                if D_delta_own_base[i,t] > D_delta_market_base[i,t]
                                    D_delta_own_base[i,t] = D_delta_market_base[i,t]
                                end
                                
                                # update A
                                
                                A_tracker_market_base[i,t+delta_t] = A_tracker_market_base[i,t] - D_market_true_base_t[i,t] + I_market_true_base_t[i,t]
                                A_tracker_own_base[i,t+delta_t] = A_tracker_own_base[i,t] - D_own_true_base_t[i,t] + I_own_true_base_t[i,t]
                            
                            end
                            
                            D_delta_market_base_total = sum(D_delta_market_base[i,t] for i in nodes)
                            D_delta_own_base_total = sum(D_delta_own_base[i,t] for i in nodes)
                            I_market_true_total = sum(I_market_true_t[i,t+delta_t] for i in nodes)
                            I_own_true_total = sum(I_own_true_t[i,t+delta_t] for i in nodes)
                            
                            # update I
                            for i in nodes
                                I_market_true_base_t[i,t+delta_t] = round(I_market_true_t[i,t+delta_t],digits=0) + round(D_delta_market_base_total * I_market_true_t[i,t+delta_t]/I_market_true_total, digits=0) #* (1+demand_factor)
                                I_own_true_base_t[i,t+delta_t] = round(I_own_true_t[i,t+delta_t],digits=0) + round(D_delta_own_base_total * I_own_true_t[i,t+delta_t]/I_own_true_total, digits=0) #* (1+demand_factor)
                            
                                if I_market_true_base_t[i,t+delta_t]<I_own_true_base_t[i,t+delta_t]
                                    I_market_true_base_t[i,t+delta_t]=I_own_true_base_t[i,t+delta_t]
                                end
                            end 
                        end
                        
                        
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### INITIALIZE START DISTRIBUTION OF FLEET (incorporating updated shares)
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        
                        # updates observed availability based on cumulative relocs and delta D_own, instantiate with observed values (or others, e.g. for counterfactual)
                        A_market_updated = Dict()
                        A_market_no_reloc = Dict()
                        A_own_updated = Dict()
                        A_own_no_reloc = Dict()
                        
                        
                        A_market_total = sum(A_market_obs[i,start_period] for i in nodes)
                        
                        A_own_total = sum(A_own_obs[i,start_period] for i in nodes)
                        share_own_total = A_own_total/A_market_total
                        
                        A_comp_total = sum(A_market_obs[i,start_period] for i in nodes)-sum(A_own_obs[i,start_period] for i in nodes)
                        share_comp_total = A_comp_total/A_market_total
                        

                        # abs val approach using added_supply_comp, added_supply_own
                        for i in nodes
                            share_own_i = A_own_obs[i,start_period]/A_own_total
                            share_comp_i = (A_market_obs[i,start_period]-A_own_obs[i,start_period])/A_comp_total
                            share_market_i = (A_market_obs[i,start_period])/A_market_total
                            
                            # if replacement use market-level shares
                            
                            if added_supply_own == -added_supply_comp

                                A_own_updated[i,start_period] = A_own_obs[i,start_period]+round(added_supply_own*share_market_i,digits=0)
                                
                                if A_own_updated[i,start_period] < 0
                                    A_own_updated[i,start_period] = 0
                                end
                                
                                if A_own_updated[i,start_period] > A_market_obs[i,start_period]
                                    A_own_updated[i,start_period] = A_market_obs[i,start_period]
                                end
                                
                                # we essentially do no change local availability, but simply replace vehicles
                                A_market_updated[i,start_period] = A_market_obs[i,start_period]#+round(added_supply_own*share_market_i,digits=0)+round(added_supply_comp*share_market_i,digits=0) 
                                A_own_no_reloc[i,start_period] = A_own_updated[i,start_period]  
                                A_market_no_reloc[i,start_period] = A_market_updated[i,start_period]
                            
                            #if expansion use own shares
                            else
                                
                                A_own_updated[i,start_period] = A_own_obs[i,start_period]+round(added_supply_own*share_own_i,digits=0)
                                A_market_updated[i,start_period] = A_market_obs[i,start_period]+round(added_supply_own*share_own_i,digits=0)+round(added_supply_comp*share_comp_i,digits=0)
                                
                                # ensure that Avail is not negative
                                if A_own_updated[i,start_period] < 0
                                    A_own_updated[i,start_period] = 0
                                end

                                if  A_market_updated[i,start_period] < 0
                                    A_market_updated[i,start_period] = 0                           
                                end
                                
                                # endure A_own < A_market
                                
                                if  A_market_updated[i,start_period] < A_own_updated[i,start_period]
                                    A_market_updated[i,start_period] = A_own_updated[i,start_period]                          
                                end
                                
                                
                                A_market_no_reloc[i,start_period] = A_market_updated[i,start_period]
                                A_own_no_reloc[i,start_period] = A_own_updated[i,start_period]
                                
                            end
                        end  
                        
                        sum_A_market_updated = sum(A_market_updated[i,start_period] for i in nodes)
                        sum_A_own_updated= sum(A_own_updated[i,start_period] for i in nodes)
                        own_share = round((sum_A_own_updated/sum_A_market_updated)*100,digits=2)
                        
                        println("Intialized with $(sum_A_market_updated) total vehicles and $(sum_A_own_updated) ($(own_share)%) own vehicles")
                        
                       
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### Adapt D_market and I_market according to new availability factors
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        
                        # prepare data
                        
                        A_market_tracker = A_market_no_reloc  # track relocations
                        A_own_tracker = A_own_no_reloc
                        A_market_base = A_market_obs
                        A_own_base = A_own_obs
                        
                        D_market_tracker_t = Dict()
                        D_delta_market_base = Dict()
                        D_own_tracker_t = Dict()
                        D_delta_own_base= Dict()
                        
                        I_delta_market = Dict()
                        I_delta_own = Dict()
                        s_own_tracker_t = Dict()
                        
                        S_own=988
                        
                        for t in data_range[1:length(data_range)-1] 
                            for i in nodes     
                                # Increase demand tracker to desired level (proportionately across network)
                                D_market_tracker_t[i,t] = round(D_market_true_t[i,t] * (1+demand_factor), digits = 0)
                               
                                if D_market_tracker_t[i,t] > A_market_tracker[i,t]+I_market_true_base_t[i,t]
                                    D_market_tracker_t[i,t] = A_market_tracker[i,t]+I_market_true_base_t[i,t]
                                end
                                
                                D_delta_market_base[i,t] = D_market_tracker_t[i,t]- D_market_true_base_t[i,t]
                                
                                s_own_tracker_t[i,t] = ((A_own_tracker[i,t]+I_own_true_base_t[i,t])/(A_market_tracker[i,t]+I_market_true_base_t[i,t]))
                                if (A_market_tracker[i,t]+I_market_true_base_t[i,t]) == 0
                                    s_own_tracker_t[i,t] = 0
                                end

                                D_own_tracker_t[i,t] = round(D_market_tracker_t[i,t] * s_own_tracker_t[i,t],digits=0)
                                
                                if D_own_tracker_t[i,t] > A_own_tracker[i,t]+I_own_true_base_t[i,t]
                                    D_own_tracker_t[i,t] = A_own_tracker[i,t]+I_own_true_base_t[i,t]
                                end

                                D_delta_own_base[i,t]  = D_own_tracker_t[i,t] - D_own_true_base_t[i,t]   

                            end

                            # collect actually allocated additional demand
                            D_delta_market_base_total = sum(D_delta_market_base[i,t] for i in nodes)
                            D_delta_own_base_total = sum(D_delta_own_base[i,t] for i in nodes)
                            
                            # find % change
                            #D_delta_market_base_perc = D_delta_market_base_total/sum(D_market_true_t[i,t] for i in nodes)
                            #D_delta_own_base_perc = D_delta_own_base_total/sum(D_own_true_t[i,t] for i in nodes)
                            
                            #println("D_delta_market_base_total: $(D_delta_market_base_total); period: $(t)")
                            #println("D_delta_own_base_total: $(D_delta_own_base_total); period: $(t)")
                            
                            # Increase true and predicted demand to desired level irrespective of whether it can be fulfilled or not (proportionately across network)
                            for i in nodes
                                D_market_true_base_t[i,t] = round(D_market_true_t[i,t] * (1+demand_factor), digits = 0)
                                
                                D_market_pred_t[i,t] = D_market_pred_t[i,t] + round(D_market_true_t[i,t] * (demand_factor), digits = 0)
                                D_market_pred_t1[i,t] = D_market_pred_t1[i,t] + round(D_market_true_t[i,t] * (demand_factor), digits = 0)
                                D_market_pred_t2[i,t] = D_market_pred_t2[i,t] + round(D_market_true_t[i,t] * (demand_factor), digits = 0)
                                D_market_pred_tn[i,t] = D_market_pred_tn[i,t] + round(D_market_true_t[i,t] * (demand_factor), digits = 0)
                                
                                D_own_true_base_t[i,t] = round(D_own_true_base_t[i,t] * (1+demand_factor), digits = 0)
                            
                            end
                            
                            # update availabilities 
                            
                            for i in nodes
                                A_market_tracker[i,t+delta_t] = A_market_tracker[i,t] - D_market_tracker_t[i,t] + I_market_true_base_t[i,t]
                                A_own_tracker[i,t+delta_t] = A_own_tracker[i,t] - D_own_tracker_t[i,t] + I_own_true_base_t[i,t]
                            end

                            # update inflows in next period by adding/removing demand deltas vs. base (that could be satisfied)
                            
                            I_market_true_base_t_total = sum(I_market_true_base_t[i,t+delta_t] for i in nodes)
                            I_own_true_base_t_total = sum(I_own_true_base_t[i,t+delta_t] for i in nodes)
                            
                            for i in nodes
                                
                                I_delta_market[i,t+delta_t] = round(D_delta_market_base_total * (I_market_true_base_t[i,t+delta_t]/I_market_true_base_t_total),digits=0)
                                
                                I_market_true_base_t[i,t+delta_t] = I_market_true_base_t[i,t+delta_t] + I_delta_market[i,t+delta_t] 
                                I_market_pred_t[i,t+delta_t] = I_market_pred_t[i,t+delta_t] + I_delta_market[i,t+delta_t]
                                I_market_pred_t1[i,t+delta_t] = I_market_pred_t1[i,t+delta_t] + I_delta_market[i,t+delta_t]
                                I_market_pred_t2[i,t+delta_t] = I_market_pred_t2[i,t+delta_t] + I_delta_market[i,t+delta_t]
                                I_market_pred_tn[i,t+delta_t] = I_market_pred_tn[i,t+delta_t] + I_delta_market[i,t+delta_t]
                                
                                
                                I_delta_own[i,t+delta_t] = round(D_delta_own_base_total * (I_own_true_base_t[i,t+delta_t]/I_own_true_base_t_total),digits=0)
                                          
                                I_own_true_base_t[i,t+delta_t] = I_own_true_base_t[i,t+delta_t] + I_delta_own[i,t+delta_t] 
                                I_own_pred_t[i,t+delta_t] = I_own_pred_t[i,t+delta_t] + I_delta_own[i,t+delta_t] 
                                I_own_pred_t1[i,t+delta_t] = I_own_pred_t1[i,t+delta_t] + I_delta_own[i,t+delta_t] 
                                I_own_pred_t2[i,t+delta_t] = I_own_pred_t2[i,t+delta_t] + I_delta_own[i,t+delta_t] 
                                I_own_pred_tn[i,t+delta_t] = I_own_pred_tn[i,t+delta_t] + I_delta_own[i,t+delta_t] 
                                
                                # update availability tracker for next period
                                
                                if I_market_true_base_t[i,t+delta_t]<I_own_true_base_t[i,t+delta_t]
                                    println("Alert 2!   I_market_true_base_t[i,t+delta_t]<I_own_true_base_t[i,t+delta_t]   ", t+delta_t)
                                end
                                
                            end
                            
                            #println("I market $(t+delta_t). ", sum(I_market_true_t[i,t+delta_t] for i in nodes))
                            #println("I own $(t+delta_t). ", sum(I_own_true_t[i,t+delta_t] for i in nodes))
                        end
                        
                        
                        # this will be used for the simulation and the perfect info case
                        I_own_updated = Dict()
                        for i in nodes
                            I_own_updated[i,start_period] = I_own_true_base_t[i,start_period]
                        end

                        # this will be used for the pred. info case
                        I_own_pred_updated = Dict()
                        for i in nodes
                            I_own_pred_updated[i,start_period] = I_own_pred_t[i,start_period]
                        end
                        

                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### Start simulation
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##

                        I_own_true_MOLO = Dict()
                        I_own_pred_MOLO = Dict()
                        I_market_true_MOLO = Dict()
                        I_market_pred_MOLO = Dict()
                        D_market_true_MOLO = Dict()
                        D_own_true_MOLO = Dict()
                        D_market_allocate_true_MOLO_no_reloc = Dict()
                        D_market_allocate_true_MOLO = Dict()
                        D_market_pred_MOLO = Dict()
                         
                        #set to simulation start
                        target_period = start_period
                        
                        #iterate until end of simuation reached
                        while target_period <= end_period   
                            
                            println("Period $(target_period) started (with $hex_res, $temporal_res ,$model, $info, $reloc_type)")
                                                        
                            times_max = [target_period, target_period+delta_t, target_period+2*delta_t, target_period+3*delta_t,target_period+4*delta_t];
                            horizon_t = times_max[1:horizon_num_periods]
                            horizon_t_max = times_max[1:2] # needed to be able to load full DFs

                            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                            #### Load data (will vary by period due to nature of multi-period prediction)
                            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                            
                            # true values
                            for i in nodes, t in horizon_t_max
                                I_own_true_MOLO[i,t] = I_own_true_base_t[i,t]
                                I_market_true_MOLO[i,t] = I_market_true_base_t[i,t]
                                D_market_true_MOLO[i,t] = D_market_true_base_t[i,t]
                                D_own_true_MOLO[i,t] = D_own_true_base_t[i,t]
                            end
                            
                            for i in nodes
                                I_own_pred_MOLO[i,target_period] = I_own_pred_t[i,target_period]
                                I_own_pred_MOLO[i,target_period+delta_t] = I_own_pred_t1[i,target_period+delta_t]
                                I_own_pred_MOLO[i,target_period+delta_t*2] = I_own_pred_t2[i,target_period+delta_t*2]
                                I_own_pred_MOLO[i,target_period+delta_t*3] = I_own_pred_tn[i,target_period+delta_t*3]
                                
                                I_market_pred_MOLO[i,target_period] = I_market_pred_t[i,target_period]
                                I_market_pred_MOLO[i,target_period+delta_t] = I_market_pred_t1[i,target_period+delta_t]
                                I_market_pred_MOLO[i,target_period+delta_t*2] = I_market_pred_t2[i,target_period+delta_t*2]
                                I_market_pred_MOLO[i,target_period+delta_t*3] = I_market_pred_tn[i,target_period+delta_t*3]

                                D_market_pred_MOLO[i,target_period] = D_market_pred_t[i,target_period]
                                D_market_pred_MOLO[i,target_period+delta_t] = D_market_pred_t1[i,target_period+delta_t]
                                D_market_pred_MOLO[i,target_period+delta_t*2]= D_market_pred_t2[i,target_period+delta_t*2]
                                D_market_pred_MOLO[i,target_period+delta_t*3] = D_market_pred_tn[i,target_period+delta_t*3]
                            end
                            
                            # fill up I_updated dictionaries for length horizon >1
                            # since the next period has already been updated in previous loop, we just need to fill the rest
                            if  length(horizon_t)>1
                                for i in nodes, t in horizon_t[2:length(horizon_t)]
                                    I_own_updated[i,t] = I_own_true_MOLO[i,t] 
                                    I_own_pred_updated[i,t] = I_own_pred_MOLO[i,t] 
                                end
                            end
                            
                            # Input values for optimization: change these depending on information case
                            if info == "PI"
                                D_market_input = D_market_true_MOLO #D_market_true_MOLO; D_own_true_MOLO
                                I_market_input =  I_market_true_MOLO #I_market_true_MOLO; I_own_updated
                                I_own_input = I_own_updated ### equal to the simulation input
                                Length_avg_input = Length_avg
                                A_market_input = A_market_updated #A_market_updated
                                A_own_input = A_own_updated
                            elseif info == "PRED"
                                D_market_input = D_market_pred_MOLO
                                I_market_input = I_market_pred_MOLO
                                I_own_input = I_own_pred_updated
                                Length_avg_input = Length_avg
                                A_market_input = A_market_updated
                                A_own_input = A_own_updated
                                
                                # adapt input based on whether period has experienced outage or not#########                                    
                                # adapt prediction if previous periods had outages
                                if (target_period-delta_t in outage_periods) == true
                                    for i in nodes
                                        D_market_input[i,target_period] = D_market_pred_t1[i,target_period]
                                        D_market_input[i,target_period+delta_t] = D_market_pred_t2[i,target_period+delta_t] # needed in dynamic model
                                        I_market_input[i,target_period] = I_market_pred_t1[i,target_period]
                                        I_market_input[i,target_period+delta_t] = I_market_pred_t2[i,target_period+delta_t]
                                    end
                                end
                                
                                # this supersedes the previous 
                                if (target_period-2*delta_t in outage_periods) == true
                                    #adjust prediction
                                    for i in nodes
                                        D_market_input[i,target_period] = D_market_pred_t2[i,target_period]
                                        D_market_input[i,target_period+delta_t] = D_market_pred_tn[i,target_period+delta_t]
                                        I_market_input[i,target_period] = I_market_pred_t2[i,target_period]
                                        I_market_input[i,target_period+delta_t] = I_market_pred_tn[i,target_period+delta_t]
                                    end
                                end                                     
                            end
        

                            ##check whether feasibility issues might occur
                            for i in nodes
                                if (A_market_input[i,target_period] + I_market_input[i,target_period])==0
                                    I_market_input[i,target_period] =1
                                    D_market_input[i,target_period] = D_market_input[i,target_period]+1
                                end
                            end
                            
                            
                            
                            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                            #### Compute re-location decisions via optimization
                            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##                        
                            
                            model_list = ("CSVP-1", "CSVP-2", "CSVP-3", "CSVP-4")

                            ##############################
                            ### OPTION 1: run NO-RELOC ###
                            ##############################    

                            if model == "NO-RELOC"
                                
                                X_results = Dict()
                                for i in nodes, j in nodes
                                    X_results[i,j,target_period] = 0
                                end
                                
                                opt_time = 0 # no optimization in NO-RELOC case

                            ##############################
                            ##### OPTION 2: run OPTIMIZATIONS #####
                            ##############################
                                
                            elseif (model in model_list) == true
                                
                                if (target_period in outage_periods) == true && response_strat == "DONOTHING"
                                    X_results = Dict()
                                    for i in nodes, j in nodes
                                        X_results[i,j,target_period] = 0
                                    end
                                    opt_time = 0 # no optimization in NO-RELOC case
                                    
                                                    
                                else # we have already adapted input based on whether period has outage or not, so can just run normally              
                                    opt_time = @elapsed X_results  = CSVP_optimization(target_period, horizon_num_periods, delta_t, nodes, nodes,
                                        D_market_input,A_market_input,I_market_input,A_own_input,I_own_input,
                                        Length_avg_input,H_max,M_max,r_var,c_var,c_reloc)  
                                    #Check infeasibility
                                    sum_X_check = sum(X_results[i,j,target_period] for i in nodes, j in nodes)

                                    if isnan(sum_X_check)
                                        println("Using myopic fallback option")
                                        X_results  = CSVP_optimization(target_period, 1, delta_t, nodes, nodes,
                                                            D_market_input,A_market_input,I_market_input,A_own_input,I_own_input,
                                                            Length_avg_input,H_max,M_max,r_var,c_var,c_reloc)
                                    end
                                end
                            else
                                println("Model not available!")
                            end
                            
                            #record time for optimization
                            opt_timer[target_period] = opt_time
                            
                            println("Optimization time: $(opt_time)")
                            
                            #Check infeasibility
                            sum_X_check = sum(X_results[i,j,target_period] for i in nodes, j in nodes)
                            if isnan(sum_X_check)
                                infeasibility_checker[target_period] = 1
                            else
                                infeasibility_checker[target_period] = 0
                            end
                            
                            # set infeasible solutions to 0 
                            if infeasibility_checker[target_period] == 1
                                for i in nodes, j in nodes  
                                    X_results[i,j,target_period] = 0
                                end
                            end
                            
                            
                            #safe re_locations
                            for i in nodes, j in nodes
                                X[i,j,target_period] = X_results[i,j,target_period]
                            end
                            
                            # compute re_locations in period  
                            sum_X = sum(X[i,j,target_period] for i in nodes, j in nodes)
                            println("Conducted re-locations in period $(target_period): $(sum_X)")                     
                            
                            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                            #### Compute results for period and updated parameters
                            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%## 
                            
                            
                            #println("start results loop")
                            for i in nodes
                                
                                s_own_pre_reloc[i,target_period] = ((A_own_no_reloc[i,target_period] + I_own_true_MOLO[i,target_period]) / (A_market_no_reloc[i,target_period] + I_market_true_MOLO[i,target_period]))
                                
                                if (A_market_no_reloc[i,target_period] + I_market_true_MOLO[i,target_period]) == 0
                                    s_own_pre_reloc[i,target_period] = 0
                                end
                                
                                # limit market demand pre reloc to actually available vehicles
                                if D_market_true_MOLO[i,target_period] > A_market_no_reloc[i,target_period] + I_market_true_MOLO[i,target_period]
                                    D_market_allocate_true_MOLO_no_reloc[i,target_period] = A_market_no_reloc[i,target_period] + I_market_true_MOLO[i,target_period]
                                    D_market_lost_no_reloc[i,target_period] = (A_market_no_reloc[i,target_period] + I_market_true_MOLO[i,target_period])-D_market_true_MOLO[i,target_period]
                                else
                                    D_market_allocate_true_MOLO_no_reloc[i,target_period] = D_market_true_MOLO[i,target_period]
                                    D_market_lost_no_reloc[i,target_period] = 0
                                end
                            
                                D_own_pre_reloc[i,target_period] = floor(D_market_allocate_true_MOLO_no_reloc[i,target_period] * s_own_pre_reloc[i,target_period] ,digits=0)
                                D_comp_pre_reloc[i,target_period] = round(D_market_allocate_true_MOLO_no_reloc[i,target_period] - D_own_pre_reloc[i,target_period],digits=0) 

                                #### CHECK!!!!
                                if D_own_pre_reloc[i,target_period] < 0
                                    println("D_own_pre_reloc negative: $(D_own_pre_reloc[i,target_period]), A_own_no_reloc = $(A_own_no_reloc[i,target_period]), I_own_true = $(I_own_true_MOLO[i,target_period])")
                                    println("D_market_true_MOLO: $(D_market_true_MOLO[i,target_period]), A_market_no_reloc: $(A_market_no_reloc[i,target_period]),I_market_true_MOLO: $(I_market_true_MOLO[i,target_period]) ]")
                                end
                                   
                                R_own_pre_reloc[i,target_period]=round(D_own_pre_reloc[i,target_period] * Length_avg_true[i,target_period]*r_var, digits=2)
                                R_comp_pre_reloc[i,target_period]=round(D_comp_pre_reloc[i,target_period] * Length_avg_true[i,target_period]*r_var, digits=2)
                                C_own_pre_reloc[i,target_period]=round(D_own_pre_reloc[i,target_period] * Length_avg_true[i,target_period]*c_var, digits=2)
                                C_comp_pre_reloc[i,target_period]=round(D_comp_pre_reloc[i,target_period] * Length_avg_true[i,target_period]*c_var, digits=2)
                                
                                 # check condition and limit re-locs to actually available vehicles (proportionally)
                                
                                if sum(X[i,j,target_period] for j in nodes) > A_own_updated[i,target_period] + I_own_updated[i,target_period]
                                    println("Reloc Alert: Not Possible")
                                    for j in nodes
                                        X[i,j,target_period] = floor((X[i,j,target_period]/sum(X[i,j,target_period] for j in nodes))*(A_own_updated[i,target_period] + I_own_updated[i,target_period]),digits=0)
                                    end
                                end
                                
                                for j in nodes  
                                    if isnan(X[i,j,target_period])
                                        X[i,j,target_period] = 0
                                    end
                                end
                                
                                
                                # add re-locations in this period
                                A_market_updated[i,target_period] = A_market_updated[i,target_period]+sum(X[j,i,target_period]-X[i,j,target_period] for j in nodes)
                                A_own_updated[i,target_period] = A_own_updated[i,target_period] +sum(X[j,i,target_period]-X[i,j,target_period] for j in nodes)
                                
                                
                                # compute demand revenue and cost post reloc, relocation in period have already been added
                                s_own_post_reloc[i,target_period] = ((A_own_updated[i,target_period] + I_own_updated[i,target_period]) / (A_market_updated[i,target_period] + I_market_true_MOLO[i,target_period]))
                                
                                if s_own_post_reloc[i,target_period]>1
                                    println(i, target_period, s_own_post_reloc[i,target_period])
                                    println("A_own: ",  A_own_updated[i,target_period])
                                    println("A_market: ",  A_market_updated[i,target_period])
                                    println("I_own: ",  I_own_updated[i,target_period])
                                    println("I_market: ",  I_market_true_MOLO[i,target_period])
                                end
                                
                                if s_own_post_reloc[i,target_period]>1
                                    s_own_post_reloc[i,target_period]=1
                                elseif s_own_post_reloc[i,target_period]<0
                                    s_own_post_reloc[i,target_period]=0
                                end
      
                                #### set s_own to 0 if nan
                                if (A_market_updated[i,target_period] + I_market_true_MOLO[i,target_period]) == 0
                                    s_own_post_reloc[i,target_period] = 0
                                end
                                
                                # limit market demand to actually available vehicles
                                if D_market_true_MOLO[i,target_period] > A_market_updated[i,target_period] + I_market_true_MOLO[i,target_period]
                                    println("Cannot fulfill D_market in tile $(i)")
                                    D_market_allocate_true_MOLO[i,target_period] = A_market_updated[i,target_period] + I_market_true_MOLO[i,target_period]
                                    D_market_lost[i,target_period] = (A_market_updated[i,target_period] + I_market_true_MOLO[i,target_period])-D_market_true_MOLO[i,target_period]
                                    println("Number of lost renatls in $(i): $(D_market_lost[i,target_period])")
                                else
                                    D_market_allocate_true_MOLO[i,target_period] = D_market_true_MOLO[i,target_period]
                                    D_market_lost[i,target_period] = 0
                                end
                                
                                # make sure own demand does not exceed availability post re-location
                                if floor(D_market_allocate_true_MOLO[i,target_period] * s_own_post_reloc[i,target_period] ,digits=0) < (A_own_updated[i,target_period] + I_own_updated[i,target_period])
                                    D_own_post_reloc[i,target_period] = floor(D_market_allocate_true_MOLO[i,target_period] * s_own_post_reloc[i,target_period] ,digits=0)
                                else
                                    D_own_post_reloc[i,target_period] = (A_own_updated[i,target_period] + I_own_updated[i,target_period])
                                end
                                
                                Delta_D_own[i,target_period] =  D_own_post_reloc[i,target_period] - D_own_pre_reloc[i,target_period]

                                # we calculate R and C based on avg. length....
                                R_own_post_reloc[i,target_period] = round(D_own_post_reloc[i,target_period]*Length_avg_true[i,target_period]*r_rent_spec,digits=2)
                                C_own_post_reloc[i,target_period] = round(D_own_post_reloc[i,target_period]*Length_avg_true[i,target_period]*c_var,digits=2)
                                
                                #if D_own_pre_reloc[i,target_period]>=D_own_post_reloc[i,target_period]
                                #    R_own_post_reloc[i,target_period] = round(D_own_post_reloc[i,target_period]*Length_own_true[i,target_period]*r_rent_spec,digits=2)
                                #elseif D_own_pre_reloc[i,target_period]<D_own_post_reloc[i,target_period]
                                #    R_own_post_reloc[i,target_period] = round((D_own_pre_reloc[i,target_period]*Length_own_true[i,target_period]+ (D_own_post_reloc[i,target_period]-D_own_pre_reloc[i,target_period])*Length_comp_true[i,target_period])*r_rent_spec,digits=2)
                                #end

                                #if D_own_pre_reloc[i,target_period]>=D_own_post_reloc[i,target_period]
                                #    C_own_post_reloc[i,target_period] = round(D_own_post_reloc[i,target_period]*Length_own_true[i,target_period]*c_var,digits=2)
                                #elseif D_own_pre_reloc[i,target_period]<D_own_post_reloc[i,target_period]
                                #    C_own_post_reloc[i,target_period] = round((D_own_pre_reloc[i,target_period]*Length_own_true[i,target_period] + (D_own_post_reloc[i,target_period]-D_own_pre_reloc[i,target_period])*Length_comp_true[i,target_period])*c_var, digits=2)
                                #end                         

                                # collect cum. net_relocs and cum demand_deltas                                
                                Cum_relocs[i,target_period] = Cum_relocs[i,target_period-delta_t] + sum(X[j,i,target_period]-X[i,j,target_period] for j in nodes)
                                Cum_Delta_D_own[i,target_period] = Cum_Delta_D_own[i,target_period-delta_t] + Delta_D_own[i,target_period]
                            
                            end
                            
                            println("D_own_pre_reloc: ",sum(D_own_pre_reloc[i,target_period] for i in nodes))
                            println("D_own_post_reloc: ",sum(D_own_post_reloc[i,target_period] for i in nodes))
                            println("Delta_D_own: ",sum(Delta_D_own[i,target_period] for i in nodes))
                            println("D_market_lost: ",sum(D_market_lost[i,target_period] for i in nodes))
                            
                            for i in nodes
                                
                                if isnan(A_market_updated[i,target_period])
                                    println("A_market_updated this period", i,target_period)
                                end
                                
                                if isnan(A_own_updated[i,target_period])
                                    println("A_own_updated this period", i,target_period)
                                end
                    
                            end
                                
                            
                            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                            #### Update Iterator and State Variables for next period
                            ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%## 
                            
                            # updated inflows
                            
                            # compute market rental gain vs. no reloc
                            
                            Delta_D_own_total = sum(Delta_D_own[i,target_period] for i in nodes)
                            D_market_true_total = sum(D_market_true_MOLO[i,target_period] for i in nodes)
                            #D_market_pred_total = sum(D_market_true_MOLO[i,target_period] for i in nodes)
                            
                            # as in the optimization we allocate based on total market inflows
                            for i in nodes
                                I_own_updated[i,target_period+delta_t] = I_own_true_MOLO[i,target_period+delta_t] + round(Delta_D_own_total * (I_market_true_MOLO[i,target_period+delta_t]/sum(I_market_true_MOLO[i,target_period+delta_t] for i in nodes)),digits=0) #round(I_market_true_MOLO[i,target_period+delta_t] * (Delta_D_own_total/D_market_true_total),digits=0)
                                
                                if I_own_updated[i,target_period+delta_t] > I_market_true_MOLO[i,target_period+delta_t]
                                    I_own_updated[i,target_period+delta_t] = I_market_true_MOLO[i,target_period+delta_t]
                                end
                                
                                I_own_pred_updated[i,target_period+delta_t] = I_own_pred_MOLO[i,target_period+delta_t] + round(Delta_D_own_total * (I_market_pred_MOLO[i,target_period+delta_t]/sum(I_market_pred_MOLO[i,target_period+delta_t] for i in nodes)),digits=0)
                                
                                if I_own_pred_updated[i,target_period+delta_t] > I_market_pred_MOLO[i,target_period+delta_t]
                                    I_own_pred_updated[i,target_period+delta_t] = I_market_pred_MOLO[i,target_period+delta_t]
                                end
                            end
                            
                            
                            println("I_market_true: ",sum(I_market_true_MOLO[i,target_period+delta_t] for i in nodes))
                            println("I_own_true: ",sum(I_own_true_MOLO[i,target_period+delta_t] for i in nodes))
                            println("I_own_updated: ",sum(I_own_updated[i,target_period+delta_t] for i in nodes))
                            
                            # allocate remaining vehicles randomly across network
                            
                            Delta_own_remain_true = round(sum(I_own_true_MOLO[i,target_period+delta_t] for i in nodes) + Delta_D_own_total - sum(I_own_updated[i,target_period+delta_t] for i in nodes),digits=0) 
                            
                            if Delta_own_remain_true > 0
                                iter = 0
                                while iter < Delta_own_remain_true 
                                    idx = nodes[rand(1:length(nodes))] #--> We use instead ordered nodes to allocate to the high-demand nodes in descending order
                                    if I_own_updated[idx,target_period+delta_t]+1 <= I_market_true_MOLO[idx,target_period+delta_t]
                                        I_own_updated[idx,target_period+delta_t] = I_own_updated[idx,target_period+delta_t] + 1
                                    end
                                    iter += 1
                                end
                            end
                            
                            #for i in nodes
                            #    if I_own_updated[i,target_period+delta_t] > I_market_true_MOLO[i,target_period+delta_t]
                            #        I_own_updated[i,target_period+delta_t] = I_market_true_MOLO[i,target_period+delta_t]
                            #    end
                            #end  
                            
                            Delta_own_remain_pred = round(sum(I_own_pred_MOLO[i,target_period+delta_t] for i in nodes) + Delta_D_own_total - sum(I_own_pred_updated[i,target_period+delta_t] for i in nodes),digits=0)
                            
                            if Delta_own_remain_true > 0
                                iter = 0
                                while iter < Delta_own_remain_pred 
                                    idx = nodes[rand(1:length(nodes))] #--> We use instead ordered nodes to allocate to the high-demand nodes in descending order
                                    if I_own_pred_updated[idx,target_period+delta_t]+1 <= I_market_pred_MOLO[idx,target_period+delta_t]
                                        I_own_pred_updated[idx,target_period+delta_t] = I_own_pred_updated[idx,target_period+delta_t] + 1
                                    end
                                    iter += 1
                                end
                            end
                            
                            #for i in nodes
                            #    if I_own_pred_updated[i,target_period+delta_t] > I_market_pred_MOLO[i,target_period+delta_t]
                            #        I_own_pred_updated[i,target_period+delta_t] = I_market_pred_MOLO[i,target_period+delta_t]
                            #    end
                            #end
                            
                            println("I_own_updated (changed): ",sum(I_own_updated[i,target_period+delta_t] for i in nodes))
                            
                            if sum(I_own_updated[i,target_period+delta_t] for i in nodes) > sum(I_market_true_MOLO[i,target_period+delta_t] for i in nodes)
                                println("I_own larger I_market!!! ", sum(I_own_updated[i,target_period+delta_t] for i in nodes), "vs",sum(I_market_true_MOLO[i,target_period+delta_t] for i in nodes))
                            end
 
                            # here we use actual data plus the cumulative net relocations to update A_market and A_own
                            for i in nodes
                                
                                # A_own here already contains re locations that have taken place during the period
                                A_market_updated[i,target_period+delta_t] = A_market_updated[i,target_period] + I_market_true_MOLO[i,target_period] - D_market_allocate_true_MOLO[i,target_period]
                                A_own_updated[i,target_period+delta_t] = A_own_updated[i,target_period] + I_own_updated[i,target_period] - D_own_post_reloc[i,target_period]
                                
                                if A_own_updated[i,target_period+delta_t] >A_market_updated[i,target_period+delta_t]
                                    A_own_updated[i,target_period+delta_t] = A_market_updated[i,target_period+delta_t]
                                end
                            
                                A_market_no_reloc[i,target_period+delta_t] = A_market_no_reloc[i,target_period] + I_market_true_MOLO[i,target_period] - D_market_allocate_true_MOLO_no_reloc[i,target_period]
                                A_own_no_reloc[i,target_period+delta_t] = A_own_no_reloc[i,target_period] + I_own_true_MOLO[i,target_period] - D_own_pre_reloc[i,target_period]
                                
                                if A_own_no_reloc[i,target_period+delta_t] >A_market_no_reloc[i,target_period+delta_t]
                                    A_own_no_reloc[i,target_period+delta_t] = A_market_no_reloc[i,target_period+delta_t]
                                end
                                
                            end                        
                         
                            for i in nodes
                                
                                if A_market_updated[i,target_period+delta_t]<0
                                    println("A_market_updated next period negative", i,target_period)
                                    println("A_market_updated (this period)", A_market_updated[i,target_period])
                                    println("I_market_true (this period)", I_market_true_MOLO[i,target_period])
                                    println("D_market_true (this period)", D_market_true_MOLO[i,target_period])
                                end
                                
                                if A_own_updated[i,target_period+delta_t]<0
                                    println("A_own_updated next period negative", i,target_period)
                                    println("A_own_updated (this period)", A_own_updated[i,target_period])
                                    println("I_own_updated (this period)", I_own_updated[i,target_period])
                                    println("D_own_post_reloc (this period)", D_own_post_reloc[i,target_period])
                                end
                    
                            end
                                                        
                            # calculate total avail
                            sum_A_market_updated = sum(A_market_updated[i,target_period+delta_t] for i in nodes)
                            sum_A_own_updated= sum(A_own_updated[i,target_period+delta_t] for i in nodes)
                            sum_A_market_no_reloc= sum(A_market_no_reloc[i,target_period+delta_t] for i in nodes)
                            sum_A_own_no_reloc= sum(A_own_no_reloc[i,target_period+delta_t] for i in nodes)
                            
                            println("Own Avail plus own inflows next period: ", sum(A_own_updated[i,target_period+delta_t] for i in nodes)+ sum(I_own_updated[i,target_period+delta_t] for i in nodes ))
                            
                            println("$(sum_A_market_updated)/$(sum_A_market_no_reloc) vehicles available with $(sum_A_own_updated)/$(sum_A_own_no_reloc) own vehicles")
                            #println("Own inflows over next period: ", sum(I_own_updated[i,target_period+delta_t] for i in nodes), "(true updated)","/",sum(I_own_pred_updated[i,target_period+delta_t] for i in nodes), "(pred updated)","/", sum(I_own_true_MOLO[i,target_period+delta_t] for i in nodes),"(no reloc)")
                            
                            # set next period
                            target_period += delta_t
                        
                        end
                                               
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### Save simulation results for case
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        
                        case = "$(info)_$(reloc_type)"
                        
                        # Re-locations (NEED TO REMOVE t DIMENSION IF COST IS TIME_INDEPENDENT)
                        Re_locations = DataFrame(Start_hex = String[],End_hex = String[],timestamp=DateTime[], Relocations = Int64[], Reloc_cost= Float64[])

                        for t in sim_range
                            for i in nodes
                                for j in nodes
                                    if c_reloc[i,j]>0
                                        append!(Re_locations, DataFrame(Start_hex = i, End_hex = j, timestamp= t, Relocations = X[i,j,t],Reloc_cost= X[i,j,t]*round(c_reloc[i,j],digits=2)))
                                    elseif c_reloc[i,j]==0
                                        append!(Re_locations, DataFrame(Start_hex = i, End_hex = j, timestamp= t, Relocations = X[i,j,t],Reloc_cost= 0))
                                    end
                                end
                            end
                        end
                        
                        CSV.write("$(save_path)$(model)_Re_locs_$(case)_$(hex_res)hex_$(temporal_res)$(suffix).csv",Re_locations)
                        
                        println("/saving relocs...")
                        
                        # Rentals
                        Rentals = DataFrame(Hex = String[],timestamp=DateTime[], Total_rentals = Int64[], 
                            Own_rentals_post_reloc = Int64[], Own_rentals_base_pre_reloc = Int64[], 
                            Own_rentals_delta = Int64[],
                            Lost_market_demand_pre = Int64[],Lost_market_demand_post = Int64[], 
                            Own_rentals_cum_delta = Int64[],Own_relocs_cum_delta=Int64[])

                        for t in sim_range
                            for i in nodes
                                append!(Rentals, DataFrame(Hex = i, timestamp= t, 
                                        Total_rentals = D_market_true_t[i,t], 
                                        Own_rentals_post_reloc = D_own_post_reloc[i,t], 
                                        Own_rentals_base_pre_reloc=D_own_pre_reloc[i,t], 
                                        Own_rentals_delta=Delta_D_own[i,t],
                                        Lost_market_demand_pre= D_market_lost_no_reloc[i,t],
                                        Lost_market_demand_post= D_market_lost[i,t],
                                        Own_rentals_cum_delta = Cum_Delta_D_own[i,t],
                                        Own_relocs_cum_delta = Cum_relocs[i,t]))
                            end
                        end
                        
                        CSV.write("$(save_path)$(model)_Rentals_$(case)_$(hex_res)hex_$(temporal_res)$(suffix).csv",Rentals)
                        
                        println("/saving rental numbers...")
                        
                        #Revenue&Cost
                        Rental_Revenue_Cost = DataFrame(Hex= String[],timestamp= DateTime[], Total_revenue = Float64[],Own_revenue_post_reloc= Float64[], Own_rental_cost_post_reloc= Float64[],
                            Own_revenue_base_pre_reloc= Float64[], Own_rental_cost_base_pre_reloc= Float64[],Comp_rental_cost_base_pre_reloc= Float64[], Own_revenue_delta = Float64[], Own_rental_cost_delta = Float64[])

                        for t in sim_range
                            for i in nodes
                                append!(Rental_Revenue_Cost, 
                                    DataFrame(
                                        Hex= i,
                                        timestamp= t, 
                                        Total_revenue = round(R_own_pre_reloc[i,t] + R_comp_pre_reloc[i,t],digits=2),
                                        Own_revenue_post_reloc= round(R_own_post_reloc[i,t],digits=2), 
                                        Own_rental_cost_post_reloc= round(C_own_post_reloc[i,t],digits=2),
                                        Own_revenue_base_pre_reloc= round(R_own_pre_reloc[i,t],digits=2),
                                        Own_rental_cost_base_pre_reloc= round(C_own_pre_reloc[i,t],digits=2), 
                                        Comp_rental_cost_base_pre_reloc = round(C_comp_pre_reloc[i,t],digits=2),
                                        Own_revenue_delta = round(R_own_post_reloc[i,t],digits=2)-round(R_own_pre_reloc[i,t],digits=2),
                                        Own_rental_cost_delta = round(C_own_post_reloc[i,t],digits=2)-round(C_own_pre_reloc[i,t],digits=2)))
                            end
                        end

                        CSV.write("$(save_path)$(model)_Revenue_Cost_$(case)_$(hex_res)hex_$(temporal_res)$(suffix).csv",Rental_Revenue_Cost)
                        
                        println("/saving revenue and cost ...")
                        
                        # Model performance
                        
                        Performance = DataFrame(timestamp= DateTime[], Optimization_time = Float64[] , Infeasibility = Int64[])
                        
                        for t in sim_range
                            append!(Performance, 
                                    DataFrame(timestamp= t,Optimization_time = opt_timer[t], Infeasibility = infeasibility_checker[t]))
                        end
                        
                        CSV.write("$(save_path)$(model)_Performance_$(case)_$(hex_res)hex_$(temporal_res)$(suffix).csv",Performance)
                        println("/saving CPU and feasibility performance ...")
                        
                        println("")
                        println("$hex_res, $temporal_res ,$model, $info, $reloc_type ended")
                        println("")
                                        
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##
                        #### Simulation for case ended (start new case)
                        ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%##

                    end # end reloc_type
                end # end info
            end # end model
        
        end # end temporal_res loop
    end # end hex_res loop

end # end function

CSVPSim (generic function with 1 method)

Run simulation

In [72]:
suffix=""
CSVPSim(suffix)

0 outage periods selected
6, 1h ,CSVP-2, PI, HUMAN started
Number of regions: 13
Intialized with 1957.0 total vehicles and 850.0 (43.43%) own vehicles
Period 2017-03-13T00:00:00 started (with 6, 1h ,CSVP-2, PI, HUMAN)
Optimization time: 1.354929737
Conducted re-locations in period 2017-03-13T00:00:00: 0.0
D_own_pre_reloc: 55.0
D_own_post_reloc: 55.0
Delta_D_own: 0.0
D_market_lost: 0
I_market_true: 108.0
I_own_true: 53.0
I_own_updated: 53.0
I_own_updated (changed): 53.0
Own Avail plus own inflows next period: 937.0
1991.0/1991.0 vehicles available with 884.0/884.0 own vehicles
Period 2017-03-13T01:00:00 started (with 6, 1h ,CSVP-2, PI, HUMAN)
Optimization time: 3.354645635
Conducted re-locations in period 2017-03-13T01:00:00: 0.0
D_own_pre_reloc: 29.0
D_own_post_reloc: 29.0
Delta_D_own: 0.0
D_market_lost: 0
I_market_true: 80.0
I_own_true: 48.0
I_own_updated: 48.0
I_own_updated (changed): 48.0
Own Avail plus own inflows next period: 956.0
2023.0/2023.0 vehicles available with 908.0/908.0

┌ Warning: NLP couldn't be solved to optimality
└ @ Juniper /Users/schroerk/.julia/packages/Juniper/8wso7/src/fpump.jl:331



Conducted re-locations in period 2017-03-15T11:00:00: 0.0
D_own_pre_reloc: 284.0
D_own_post_reloc: 286.0
Delta_D_own: 2.0
D_market_lost: 0
I_market_true: 602.0
I_own_true: 314.0
I_own_updated: 315.0
I_own_updated (changed): 316.0
Own Avail plus own inflows next period: 1090.0
1609.0/1609.0 vehicles available with 774.0/784.0 own vehicles
Period 2017-03-15T12:00:00 started (with 6, 1h ,CSVP-2, PI, HUMAN)
Academic license - for non-commercial use only - expires 2021-10-07
Optimization time: 5.055662148
Conducted re-locations in period 2017-03-15T12:00:00: 13.0
D_own_pre_reloc: 316.0
D_own_post_reloc: 322.0
Delta_D_own: 6.0
D_market_lost: 0
I_market_true: 652.0
I_own_true: 313.0
I_own_updated: 318.0
I_own_updated (changed): 319.0
Own Avail plus own inflows next period: 1087.0
1561.0/1561.0 vehicles available with 768.0/782.0 own vehicles
Period 2017-03-15T13:00:00 started (with 6, 1h ,CSVP-2, PI, HUMAN)
Academic license - for non-commercial use only - expires 2021-10-07
Optimization time

┌ Warning: NLP couldn't be solved to optimality
└ @ Juniper /Users/schroerk/.julia/packages/Juniper/8wso7/src/fpump.jl:331


Optimization time: 1.877795558
Conducted re-locations in period 2017-03-18T23:00:00: 0.0
D_own_pre_reloc: 288.0
D_own_post_reloc: 295.0
Delta_D_own: 7.0
D_market_lost: 0
I_market_true: 611.0
I_own_true: 276.0
I_own_updated: 283.0
I_own_updated (changed): 283.0
Own Avail plus own inflows next period: 1055.0
1688.0/1688.0 vehicles available with 772.0/792.0 own vehicles
Period 2017-03-19T00:00:00 started (with 6, 1h ,CSVP-2, PI, HUMAN)
Optimization time: 1.040857048
Conducted re-locations in period 2017-03-19T00:00:00: 0.0
D_own_pre_reloc: 217.0
D_own_post_reloc: 222.0
Delta_D_own: 5.0
D_market_lost: 0
I_market_true: 408.0
I_own_true: 186.0
I_own_updated: 191.0
I_own_updated (changed): 191.0
Own Avail plus own inflows next period: 1024.0
1813.0/1813.0 vehicles available with 833.0/851.0 own vehicles
Period 2017-03-19T01:00:00 started (with 6, 1h ,CSVP-2, PI, HUMAN)
Optimization time: 1.870712019
Conducted re-locations in period 2017-03-19T01:00:00: 0.0
D_own_pre_reloc: 150.0
D_own_post_r

...

Academic license - for non-commercial use only - expires 2021-10-07
Optimization time: 12.496103991
Conducted re-locations in period 2017-03-15T11:00:00: 3.0
D_own_pre_reloc: 284.0
D_own_post_reloc: 294.0
Delta_D_own: 10.0
D_market_lost: 0
I_market_true: 602.0
I_own_true: 314.0
I_own_updated: 323.0
I_own_updated (changed): 324.0
Own Avail plus own inflows next period: 1084.0
1623.0/1609.0 vehicles available with 760.0/784.0 own vehicles
Period 2017-03-15T12:00:00 started (with 6, 1h ,CSVP-2, PI, HYBRID)
Academic license - for non-commercial use only - expires 2021-10-07
Optimization time: 3.239204671
Conducted re-locations in period 2017-03-15T12:00:00: 6.0
D_own_pre_reloc: 316.0
D_own_post_reloc: 326.0
Delta_D_own: 10.0
D_market_lost: 0
I_market_true: 652.0
I_own_true: 313.0
I_own_updated: 322.0
I_own_updated (changed): 323.0
Own Avail plus own inflows next period: 1081.0
1575.0/1561.0 vehicles available with 758.0/782.0 own vehicles
Period 2017-03-15T13:00:00 started (with 6, 1h ,CSV

┌ Warning: NLP couldn't be solved to optimality
└ @ Juniper /Users/schroerk/.julia/packages/Juniper/8wso7/src/fpump.jl:331


Academic license - for non-commercial use only - expires 2021-10-07
Optimization time: 8.016647968
Conducted re-locations in period 2017-03-15T14:00:00: 5.0
D_own_pre_reloc: 370.0
D_own_post_reloc: 384.0
Delta_D_own: 14.0
D_market_lost: 0
I_market_true: 804.0
I_own_true: 363.0
I_own_updated: 375.0
I_own_updated (changed): 377.0
Own Avail plus own inflows next period: 1085.0
1485.0/1469.0 vehicles available with 708.0/736.0 own vehicles
Period 2017-03-15T15:00:00 started (with 6, 1h ,CSVP-2, PI, HYBRID)
Optimization time: 0.953680889
Conducted re-locations in period 2017-03-15T15:00:00: 1.0
D_own_pre_reloc: 376.0
D_own_post_reloc: 386.0
Delta_D_own: 10.0
D_market_lost: 0
I_market_true: 853.0
I_own_true: 397.0
I_own_updated: 407.0
I_own_updated (changed): 407.0
Own Avail plus own inflows next period: 1106.0
1498.0/1482.0 vehicles available with 699.0/723.0 own vehicles
Period 2017-03-15T16:00:00 started (with 6, 1h ,CSVP-2, PI, HYBRID)
Academic license - for non-commercial use only - exp

┌ Warning: NLP couldn't be solved to optimality
└ @ Juniper /Users/schroerk/.julia/packages/Juniper/8wso7/src/fpump.jl:331



Conducted re-locations in period 2017-03-19T00:00:00: 1.0
D_own_pre_reloc: 217.0
D_own_post_reloc: 222.0
Delta_D_own: 5.0
D_market_lost: 0
I_market_true: 408.0
I_own_true: 186.0
I_own_updated: 191.0
I_own_updated (changed): 191.0
Own Avail plus own inflows next period: 1017.0
1836.0/1813.0 vehicles available with 826.0/851.0 own vehicles
Period 2017-03-19T01:00:00 started (with 6, 1h ,CSVP-2, PI, HYBRID)
Optimization time: 1.267149107
Conducted re-locations in period 2017-03-19T01:00:00: 0.0
D_own_pre_reloc: 150.0
D_own_post_reloc: 153.0
Delta_D_own: 3.0
D_market_lost: 0
I_market_true: 286.0
I_own_true: 143.0
I_own_updated: 145.0
I_own_updated (changed): 145.0
Own Avail plus own inflows next period: 1009.0
1900.0/1877.0 vehicles available with 864.0/887.0 own vehicles
Period 2017-03-19T02:00:00 started (with 6, 1h ,CSVP-2, PI, HYBRID)
Optimization time: 1.206136205
Conducted re-locations in period 2017-03-19T02:00:00: 0.0
D_own_pre_reloc: 96.0
D_own_post_reloc: 99.0
Delta_D_own: 3.0
D

---